This tutorial will show how to use `mhar` for sampling polytopes. It is focused on executing parallel MCMC walks over a polytope in GPUs.

In [1]:
import torch

ModuleNotFoundError: No module named 'torch'

Before starting let's check if you have an avaialble gpu device or not.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type
print('Device:', device)

Device: cuda


We also need to decide the data-type `dtype` we are going to use. Depending on your necessities you can choose it, we recomend to use `64` bits for non-fully dimentional polytopes in order to maintain numerical inestability of the projections. Otherwiise the precision depends on the dimension of your polytope and speed you want.  
  
As of now `16` bit precision is only available for `gpu` and not `cpu`.

In [ ]:
# We will choose 64-bits
dtype = torch.float64

## Canonical Representation

The polytope in question must be presented in matrix canonical representation (as opposed to vertex). `mhar` assumes that the matrix has no repeated or redundant restrictions.

## Non-Fully dimensional Polytopes  

### Definition

> $A^IX \leq b^I$  
> 
> $A^EX = b^E$


For non-fully dimensional polytopes we need to use the class `NFDPolytope` in the `mhar.polytope` module. The restrictions must be passed as pytorch tensors.  
  
We will sample the unit hypercube that is defined as:  
> $n-simplex = \{x \in R^n || \sum_{i=1}^{n} x_i = 1, 0 \leq x_i \} $  

Which we can represent in matrix restrictions:  
$ -Ix \leq 0$  
$ [1]^n = 1 $  
Where $I$ is the identity matrix of dimension $n \times n$ 


#### Definition-Code

Lets create the tensors to represent the restrictions that define the polytope. Since we need to create a projection matrix for the non-fully dimensional object we need to preserve the numerical stability of the algorithm, we suggest using 64 bits precision. 

In [ ]:
n = 3 # Dimension
dtype = torch.float64 # Precision 
A_I = torch.eye(n).to(dtype) * -1.0
b_I = torch.empty(n, 1, dtype=dtype)
b_I.fill_(0.0)

# Create Equalities
A_E = torch.empty(1, n, dtype=dtype)
A_E.fill_(1.0)
b_E = torch.empty(1, 1, dtype=dtype)
b_E.fill_(1.0)      
print(f'Inequality Matrix A^I \n {A_I} \n')
print(f'Inequality Vector b^I \n {b_I}\n')
print(f'Equality Matrix A^E \n {A_E} \n')
print(f'Equality Vector b^E \n {b_E}')

Inequality Matrix A^I 
 tensor([[-1., -0., -0.],
        [-0., -1., -0.],
        [-0., -0., -1.]], dtype=torch.float64) 

Inequality Vector b^I 
 tensor([[0.],
        [0.],
        [0.]], dtype=torch.float64)

Equality Matrix A^E 
 tensor([[1., 1., 1.]], dtype=torch.float64) 

Equality Vector b^E 
 tensor([[1.]], dtype=torch.float64)


Now lets create a `NFDPolytope` object to represent the polytope.

In [ ]:
from mhar.polytope import NFDPolytope
simplex = NFDPolytope(A_I, # Inequality Restriction Matrix 
                     b_I,  # Inequality Vector
                     A_E, # Equality Restriction Matrix 
                     b_E,  # Equality Vector
                     dtype, # torch dtype
                     device, # device used cpu or cuda
                     copy=False # bool for creating a copy of the restrictions
                     )

/home/uumami/sonder.art/mhar/mhar/polytope.py:45: UserWarning:
  The object will not create a copy of the tensors, so modifications will be reflected in the object



In [ ]:
simplex

Numeric Precision (dtype) torch.float64
Device: cuda
A_in: torch.Size([3, 3]) 
b_in: torch.Size([3, 1])
A_eq: torch.Size([1, 3]) 
b_eq: torch.Size([1, 1])

### Projection Matrix

Now we need to compute que projection matrix that we will use for projecting the random directions vectors to the equality space. For that we can use the method `NFDPolytope.compute_projection_matrix()`. We recommend using the highest precision possible to compute this matrix.

In [ ]:
simplex.compute_projection_matrix(device=device, solver_precision=torch.float64)

Max non zero error for term (A A')^(-1)A at precision torch.float64:  tensor(0., device='cuda:0', dtype=torch.float64)


In [ ]:
simplex

Numeric Precision (dtype) torch.float64
Device: cuda
A_in: torch.Size([3, 3]) 
b_in: torch.Size([3, 1])
A_eq: torch.Size([1, 3]) 
b_eq: torch.Size([1, 1])
Projection Matrix: torch.Size([3, 3])

### Starting Inner Point(s)

In order to start the algorithm we need at least one inner point $x_0$. If you know your inner point you can supply it to the algorithm, `mhar` also contains functions to compute one inner point using the [chebyshev center](https://en.wikipedia.org/wiki/Chebyshev_center) which finds the center of the smallest ball inside the polytope.

 `from mhar.inner_point import ChebyshevCenter`. The solver is in numpy so precision must be specified as `numpy.dtype`. It uses `linprog` from `scipy.optimize`. You can see the documentation [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html). 
  
It could also be the last points produced by a previous walk/run of the `mhar` 

In [ ]:
from mhar.inner_point import ChebyshevCenter
import numpy as np

In [ ]:
x0 = ChebyshevCenter(polytope=simplex, # Polytope Object
                    lb=None,  # Lowerbound (lb <= x ), if unknown leave it as None 
                    ub=None,  # Upperbound ( x <= up), if unknown leave it as None 
                    tolerance=1e-4, # Tolerance for equality restrictions (A_eqx = b_eq)
                    device='cuda', # device used cpu or cuda
                    solver_precision=np.float64 # numpy dtype
                    )


Simplex Status for the Chebyshev Center
 Optimization proceeding nominally.


In [ ]:
x0

tensor([[0.3333],
        [0.3333],
        [0.3333]], device='cuda:0', dtype=torch.float64)

If we want to manually input the inner points then it is enough to use a torch tensor of size $n \times l$. Where $l$ is ne number of inner points you want to supply. Just write them in column notation.  
  
We are going to manually add an other starting point to the one calcualted by the `chebyshev center` to show its functionality later.

In [ ]:
x0 = torch.cat([x0, 
            torch.tensor([[.25], [.25], [.5]]).to(device).to(dtype)
             ], dim=1)
x0

tensor([[0.3333, 0.2500],
        [0.3333, 0.2500],
        [0.3333, 0.5000]], device='cuda:0', dtype=torch.float64)

Now we can proceed to sample the `polytope`

### Walk

We are going to sample the polytope starting from the inner points we supply using the method `walk.walk`. It has the next arguments:

+ `polytope` is an object of the type `Polytope` or `NFDPolytope` that defines it.
+ `X0` a tensor containing the inner points to start the walks from.
+ `z` determines the number of simoultaneous `walks`. If the number of initial points supplied are less than `z`  ($ncols($ `x0` $) < $ `z`) then some points will be reused as starting points.  
+ `T` is the number of uncorrelated iterations you want. The number of total uncorrelated points produced by the algorithm is `z` $\times$ `T`, since `z` points are sampled at each iteration.  
+ `thinning` determines the number of points that we need to burn between iterations in order to get uncorrelated points. The suggested factor should be in the order of $O(n^3)$.
+ `warm` determines a thinning for warming the walks only at the beggining, after the this war the walks resumes as normal. It is used if you want to lose the dependency from the starting points.
+ `device` device where the tenros live `cpu` or `cuda`
+ `seed` for reproducibility
+ `verbosity` for printing what is going on

In [ ]:
x0

tensor([[-0.],
        [-0.],
        [-0.],
        [-0.],
        [-0.],
        [-0.],
        [-0.],
        [-0.],
        [-0.],
        [-0.]], device='cuda:0')

In [ ]:
from mhar.walk import walk
X = walk(polytope=simplex,
        X0 = x0,  
        z=100, 
        T=10, 
        warm=0,
        thinning=n**3, 
        device=device, 
        seed=None,
        verbosity=2
)

AssertionError: 

`walk` produces `T` $\times$ `z` uncorrelated points. It returns a vector of dimension `T` $\times$ `z` $\times$ `n`.  

In [ ]:
X

tensor([[[ 1.4841e-01,  6.8280e-01,  4.5665e-02, -5.0482e-01,  8.0002e-01,
           8.4413e-01,  7.0327e-01,  8.1021e-01, -5.8382e-02,  8.6097e-01,
          -4.0612e-01, -1.5000e-01, -7.2603e-01,  9.1046e-01,  5.8878e-01,
           4.3969e-01, -8.7148e-01,  2.2590e-01,  4.2120e-01,  8.2291e-01,
          -4.3603e-01, -8.6074e-01,  4.7400e-01, -5.4297e-01,  9.2238e-02,
           1.3477e-01,  8.7554e-01,  1.8995e-01,  6.9708e-01,  2.0776e-01,
          -3.8575e-01,  8.0821e-01,  7.2754e-01, -7.9290e-01, -5.0327e-01,
           3.3527e-01, -4.2698e-01,  7.7645e-01, -1.9216e-01,  7.3043e-02,
           2.5659e-01,  3.5802e-01, -1.4228e-01, -3.6713e-01, -1.7050e-01,
          -7.3152e-01, -1.6744e-01,  1.9559e-01,  7.7014e-01, -7.0855e-01,
           6.1163e-01,  7.4568e-01,  4.3088e-01,  9.6311e-01, -2.0932e-01,
          -1.2013e-01,  1.1033e-01,  1.1165e-02, -1.2734e-01,  6.2316e-01,
           4.2978e-01,  4.6269e-01, -1.4436e-01,  1.8806e-01, -3.2818e-01,
          -7.5972e-01,  7

In [ ]:
X.sum(1)

tensor([[1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1

In [ ]:
X.shape

torch.Size([10, 3, 100])

### Numerical Stability

In [ ]:
print('Infinite: ', (~torch.isfinite(X)).sum())
print('Nans:  ',(torch.isnan(X)).sum())
print('Inequality violation:  ',(X.sum(1)!=1.0).sum())
print('Inequality violation with tol:  ',((X.sum(1)- 1.0).abs()>1e-6).sum())

Infinite:  tensor(0)
Nans:   tensor(0)
Inequality violation:   tensor(100)
Inequality violation with tol:   tensor(100)


### Summary

To sumamrize the steps taken we can use the `polytope_examples` for creating a `Hypercube`.

In [ ]:
from mhar.polytope_examples import Simplex

# Create a polytope (Simplex)
hypercube = Simplex(10,
                      dtype=torch.float32,
                      device='cuda'
                      )

/home/uumami/sonder.art/mhar/mhar/polytope.py:45: UserWarning:
  The object will not create a copy of the tensors, so modifications will be reflected in the object



Define/Find inner points

In [ ]:
x0 = ChebyshevCenter(polytope=hypercube, 
                    lb=None, 
                    ub=None, 
                    tolerance=1e-4,
                    device='cuda',
                    solver_precision=np.float32)




Simplex Status for the Chebyshev Center
 Optimization proceeding nominally.


Sample points

In [ ]:
X = walk(polytope=hypercube,
        X0 = x0,  
        z=100, 
        T=1, 
        warm=0,
        thinning=10000, 
        device= None, 
        seed=None,
        verbosity=2
)
X

Minimum number allowed -3.4028234663852886e+38
Maximum number allowed 3.4028234663852886e+38
Eps:  1.1920928955078125e-07
Values close to zero will be converted to 3eps or -3eps: 3.5762786865234375e-07
n:  10   mI: 20   mE: None   z: 100


% of burned samples |██████████████████████████████| 100.0%
% of iid samples |██████████████████████████████| 100.0%


tensor([[[ 1.4841e-01,  6.8280e-01,  4.5665e-02, -5.0482e-01,  8.0002e-01,
           8.4413e-01,  7.0327e-01,  8.1021e-01, -5.8382e-02,  8.6097e-01,
          -4.0612e-01, -1.5000e-01, -7.2603e-01,  9.1046e-01,  5.8878e-01,
           4.3969e-01, -8.7148e-01,  2.2590e-01,  4.2120e-01,  8.2291e-01,
          -4.3603e-01, -8.6074e-01,  4.7400e-01, -5.4297e-01,  9.2238e-02,
           1.3477e-01,  8.7554e-01,  1.8995e-01,  6.9708e-01,  2.0776e-01,
          -3.8575e-01,  8.0821e-01,  7.2754e-01, -7.9290e-01, -5.0327e-01,
           3.3527e-01, -4.2698e-01,  7.7645e-01, -1.9216e-01,  7.3043e-02,
           2.5659e-01,  3.5802e-01, -1.4228e-01, -3.6713e-01, -1.7050e-01,
          -7.3152e-01, -1.6744e-01,  1.9559e-01,  7.7014e-01, -7.0855e-01,
           6.1163e-01,  7.4568e-01,  4.3088e-01,  9.6311e-01, -2.0932e-01,
          -1.2013e-01,  1.1033e-01,  1.1165e-02, -1.2734e-01,  6.2316e-01,
           4.2978e-01,  4.6269e-01, -1.4436e-01,  1.8806e-01, -3.2818e-01,
          -7.5972e-01,  7